<a href="https://colab.research.google.com/github/gabrielalonde/ProjetoPIBIC/blob/main/ProjetoPIBIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Trasforma arquivo CSV em FASTA

In [ ]:
import csv
csv_nome = 'grupo+.csv'  # nome do arquivo CSV
fasta_nome = 'grupo+.fasta'  #nome do arquivo FASTA
with open(csv_nome, 'r') as csv_file, open(fasta_nome, 'w') as fasta_file:  #abre os arquivos
    csv_reader = csv.reader(csv_file)
    header = next(csv_reader) #ignora o cabeçalho

    for row in csv_reader:
        # Cabecalho das colunas do arquivo CSV
        ID = row[0]
        sequence = row[1]
        atividade = row[2]
        banco = row[3]
        tamanho = row[4]

        # escreve a sequência no arquivo FASTA
        fasta_file.write('>' + ID + '|' + atividade + '|' + banco + '|' + tamanho + '\n')
        fasta_file.write(sequence + '\n')```


Trasforma arquivo FASTA em CSV

In [ ]:
### TRANSFORMA UM ARQUIVO FASTA EM CSV
import csv

csv_nome = 'dados+.csv'
fasta_nome = 'grupo+.fasta'
csv_header = ['ID', 'sequence', 'atividade', 'banco', 'tamanho']

with open(fasta_nome, 'r') as fasta_file, open(csv_nome, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(csv_header)

    ID = ''
    sequence = ''
    atividade = ''
    banco = ''
    tamanho = ''

    for line in fasta_file:
        if line.startswith('>'):
            if ID:
                csv_writer.writerow([ID, sequence, atividade, banco, tamanho])

            fields = line.strip().lstrip('>').split('|')
            ID = fields[0]
            atividade = fields[1]
            banco = fields[2]
            tamanho = fields[3]
            sequence = ''
        else:
            sequence += line.strip()

    csv_writer.writerow([ID, sequence, atividade, banco, tamanho])


Filtragem dos aminoácidos

In [ ]:
import csv

with open('negativo.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader)
    results = []
    for row in csv_reader:
        ID = row[0]
        sequencia = row[1]
        atividade = row[2]
        banco = row[3]
        tamanho = row[4]
        if not any(x in sequencia for x in ['Abu', 'O', 'U', 'Bal', 'Nal', 'X']):
            results.append([ID, sequencia, atividade, banco, tamanho])

with open('consolidado-filtrado1.csv', mode='w', newline='') as results_file:
    writer = csv.writer(results_file)
    writer.writerow(['ID', 'Sequência', 'atividade','banco','tamanho'])
    for result in results:
        writer.writerow(result)


Removendo as duplicatas

In [ ]:
## RETIRA AS SEQUENCIAS DUPLICADAS DE UMA MESMA COLUNA ##

import pandas as pd
df = pd.read_csv('arquivo.csv')
df_unique = df.drop_duplicates(subset='Sequência') #REMOVE AS DUPLICATAS DA COLUNA SEQUENCIA
df_unique.to_csv('arquivo_+sem_duplicatas.csv', index=False)


Peptide Descriptor

In [ ]:
pip install modlamp

In [ ]:
import csv
from modlamp.descriptors import GlobalDescriptor, PeptideDescriptor


In [ ]:
rows = []

with open('grupo+.csv', 'r') as file:
    reader = csv.reader(file)
    header = next(reader)

    # Adicionar nome as novas colunas
    header.extend(['NetCharge', 'BomanIndex', 'HR', 'Aliphatic', 'II', 'IP', 'HM'])

    # Percorrer as linhas
    for row in reader:
        # Extrair a sequência de peptídeo da coluna peptideo
        peptide = row[1]

        #NetCharge
        desc = GlobalDescriptor(peptide)
        desc.calculate_charge(ph=7.4)
        netcharge = desc.descriptor[0][0]

        #BomanIndex
        try:
            desc.boman_index()
        except:
            breakpoint()
        bomanIndex = desc.descriptor[0][0]

        #HydrophobicRatio
        desc.hydrophobic_ratio()
        HR = desc.descriptor[0][0]

        #AliphaticIndex
        desc.aliphatic_index()
        al = desc.descriptor[0][0]

        #InstabilityIndex
        desc.instability_index()
        II = desc.descriptor[0][0]

        #IsoelectricPoint
        desc.isoelectric_point()
        IP = desc.descriptor[0][0]

        #HydrophobicMoment
        desc2 = PeptideDescriptor(peptide, "eisenberg")
        desc2.calculate_moment(window=1000)
        HM = desc2.descriptor[0][0]

        row.extend([netcharge, bomanIndex, HR, al, II, IP, HM])

        rows.append(row)

# Escrever os dados atualizados em um novo arquivo CSV
with open('grupo+results.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(rows)  # Escrever as linhas atualizadas
